In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from pathlib import Path
import joblib
import warnings
import logging
from datetime import datetime
from typing import Tuple, Optional, Dict
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
sns.set_theme(style="whitegrid")

# -------------------------------------------------------
# Setup Logging
# -------------------------------------------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# -------------------------------------------------------
# Paths
# -------------------------------------------------------
# *** IMPORTANT: This path must be correct for the script to run ***
FEATURE_PATH = Path("data/raw/nepal_electronics_transactions_6000_with_names.csv")

OUT_DIR = Path("data/processed")
MODEL_DIR = Path("models")
CACHE_PATH = MODEL_DIR / "txn_matrix_cache.pkl"
RULES_PATH = MODEL_DIR / "apriori_rules.pkl"

# -------------------------------------------------------
# 1. Data Quality & Validation
# -------------------------------------------------------
def validate_data(df: pd.DataFrame) -> pd.DataFrame:
    logger.info("Validating input data...")
    required_cols = ['transaction_id', 'product_name']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    df = df.dropna(subset=['transaction_id', 'product_name'])
    df = df[df['product_name'].str.strip() != '']
    if len(df) < 10:
        raise ValueError("Insufficient data: need at least 10 transactions")
    logger.info(f"✓ Data validation complete: {len(df)} rows validated")
    return df

def print_data_quality_report(df: pd.DataFrame):
    print("\n" + "="*70)
    print("📊 DATA QUALITY REPORT")
    print("="*70)
    print(f"Total rows: {len(df):,}")
    print(f"Unique transactions: {df['transaction_id'].nunique():,}")
    print(f"Unique products: {df['product_name'].nunique():,}")
    items_per_txn = df.groupby('transaction_id').size()
    print(f"Avg items per transaction: {items_per_txn.mean():.2f}")
    product_freq = df['product_name'].value_counts()
    print(f"Most common product: {product_freq.index[0]} ({product_freq.iloc[0]} times)")
    if 'category' in df.columns:
         print(f"Unique categories: {df['category'].nunique():,}")
    print("="*70 + "\n")

# -------------------------------------------------------
# 2. Build Transaction Matrix
# -------------------------------------------------------
def create_transaction_matrix(df: pd.DataFrame, use_cache: bool = True) -> pd.DataFrame:
    if use_cache and CACHE_PATH.exists():
        logger.info("Loading cached transaction matrix...")
        try:
            return joblib.load(CACHE_PATH)
        except Exception as e:
            logger.warning(f"Cache load failed: {e}. Rebuilding matrix...")
    logger.info("Building transaction matrix...")
    df["qty"] = 1
    txn = df.pivot_table(index="transaction_id", columns="product_name", values="qty", aggfunc="sum", fill_value=0)
    txn = txn.astype(bool)
    if use_cache:
        MODEL_DIR.mkdir(parents=True, exist_ok=True)
        joblib.dump(txn, CACHE_PATH)
        logger.info("✓ Transaction matrix cached")
    return txn

# -------------------------------------------------------
# 3. Run Apriori + Association Rules
# -------------------------------------------------------
def run_apriori(txn_matrix: pd.DataFrame, min_support: float = 0.01, min_confidence: float = 0.2, max_len: Optional[int] = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
    logger.info(f"Running Apriori (support={min_support}, confidence={min_confidence})...")
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", DeprecationWarning)
        frequent_itemsets = apriori(txn_matrix, min_support=min_support, max_len=max_len, use_colnames=True)
    if len(frequent_itemsets) == 0:
        logger.warning("No frequent itemsets found. Try lowering 'min_support'.")
        return pd.DataFrame(), pd.DataFrame()
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    logger.info(f"✓ Found {len(frequent_itemsets)} itemsets, {len(rules)} rules")
    return frequent_itemsets, rules

# -------------------------------------------------------
# 4. Hierarchical Product + Category Recommendations
# -------------------------------------------------------
def generate_hierarchical_recommendations(df: pd.DataFrame, rules: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if 'category' not in df.columns or len(rules) == 0:
        logger.warning("Skipping hierarchical recommendations: 'category' column not found.")
        return pd.DataFrame(), pd.DataFrame()
    prod_cat_map = df.groupby('product_name')['category'].first().to_dict()
    reco = []
    cat_summary = {}
    for _, row in rules.iterrows():
        ant_list = list(row['antecedents'])
        cons_list = list(row['consequents'])
        for p_from in ant_list:
            for p_to in cons_list:
                cat_from = prod_cat_map.get(p_from)
                cat_to = prod_cat_map.get(p_to)
                if not cat_from or not cat_to:
                    continue
                type_assoc = 'same_category' if cat_from == cat_to else 'cross_category'
                reco.append({
                    "product": p_from,
                    "recommended_product": p_to,
                    "product_category": cat_from,
                    "recommended_category": cat_to,
                    "type": type_assoc,
                    "lift": row['lift'],
                    "confidence": row['confidence'],
                    "support": row['support']
                })
                # Category summary
                key = (cat_from, cat_to)
                if key not in cat_summary:
                    cat_summary[key] = []
                cat_summary[key].append(row['lift'])
    product_category_reco = pd.DataFrame(reco)
    # Category summary
    summary_rows = []
    for (cat_from, cat_to), lifts in cat_summary.items():
        avg_lift = np.mean(lifts)
        summary_rows.append({
            "category_from": cat_from,
            "category_to": cat_to,
            "num_product_pairs": len(lifts),
            "avg_lift": avg_lift
        })
    category_summary = pd.DataFrame(summary_rows).sort_values('avg_lift', ascending=False)
    return category_summary, product_category_reco

# -------------------------------------------------------
# 5. Run Full Pipeline
# -------------------------------------------------------
def run_apriori_pipeline(min_support=0.005, min_confidence=0.05, max_len=None, use_cache=True):
    try:
        logger.info("="*70)
        logger.info("🚀 STARTING ENHANCED APRIORI PIPELINE")
        logger.info("="*70)
        df = pd.read_csv(FEATURE_PATH)
        df = validate_data(df)
        print_data_quality_report(df)
        txn = create_transaction_matrix(df, use_cache)
        frequent_itemsets, rules = run_apriori(txn, min_support, min_confidence, max_len)
        category_summary, product_category_reco = generate_hierarchical_recommendations(df, rules)
        
        logger.info("✅ PIPELINE COMPLETED SUCCESSFULLY")
        return {
            'rules': rules,
            'category_summary': category_summary,
            'product_category_reco': product_category_reco
        }
    except FileNotFoundError:
        logger.error(f"❌ Pipeline failed: Input file not found at {FEATURE_PATH}")
        return None
    except Exception as e:
        logger.error(f"❌ Pipeline failed: {e}")
        return None

# -------------------------------------------------------
# 6. Run the Pipeline
# -------------------------------------------------------
# Adjust min_support if you get "No frequent itemsets found"
results = run_apriori_pipeline(
    min_support=0.005,  # 0.5% of all transactions
    min_confidence=0.05, # 5% confidence
    use_cache=False      # Set to False to rebuild matrix
)

# -------------------------------------------------------
# 7. Analyze and Visualize Results (THE DIAGRAMS)
# -------------------------------------------------------
if results and not results['rules'].empty:
    rules = results['rules']
    category_summary = results['category_summary']

    print("\n\n" + "="*70)
    print("ANALYSIS: PLOTTING APRIORI DIAGRAMS")
    print("="*70)
    
    print("Top 20 Rules by Lift:")
    display(rules.sort_values('lift', ascending=False).head(20))

    # --- Diagram 1: Rule Scatter Plot ---
    print("\n--- Diagram 1: Rule Scatter Plot (Support vs. Confidence) ---")
    plt.figure(figsize=(12, 7))
    sns.scatterplot(
        x='support', 
        y='confidence', 
        size='lift', 
        data=rules[rules['lift'] > 1.5],  # Only plot rules with lift > 1.5
        hue='lift',
        palette='viridis',
        sizes=(20, 500),
        alpha=0.7
    )
    plt.title('Association Rules (Lift > 1.5)\nSupport vs. Confidence, Sized by Lift')
    plt.xlabel('Support')
    plt.ylabel('Confidence')
    plt.legend(title='Lift', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # --- Diagram 2: Category Heatmap ---
    if not category_summary.empty:
        print("\n--- Diagram 2: Category-to-Category Lift Heatmap ---")
        
        # Pivot for heatmap
        category_pivot = category_summary.pivot_table(
            index='category_from', 
            columns='category_to', 
            values='avg_lift',
            aggfunc='mean'
        )
        
        # Limit to top N categories for readability
        top_cats_from = category_summary['category_from'].value_counts().index[:15]
        top_cats_to = category_summary['category_to'].value_counts().index[:15]
        
        category_pivot_filtered = category_pivot.loc[
            category_pivot.index.isin(top_cats_from),
            category_pivot.columns.isin(top_cats_to)
        ]

        plt.figure(figsize=(16, 12))
        sns.heatmap(
            category_pivot_filtered, 
            annot=True, 
            fmt=".2f", 
            cmap="viridis", 
            linewidths=0.5,
            cbar_kws={'label': 'Average Lift'}
        )
        plt.title('Heatmap of Average Lift Between Top Product Categories')
        plt.xlabel('Recommended Category ("Consequent")')
        plt.ylabel('Original Category ("Antecedent")')
        plt.tight_layout()
        plt.show()
    else:
        print("Skipping category heatmap (no category data found or no rules generated).")

else:
    print("\n--- No results to plot. ---")
    print("This likely means 'min_support' was too high and no rules were found.")
    print("Try lowering 'min_support' in the 'run_apriori_pipeline' call (e.g., to 0.001).")